In [ ]:
import numpy as np
import pandas as pd
import os
import zipfile
import shutil
import tempfile

# Настраиваем отображение всех столбцов
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 10)

# Путь к папке с файлами
drive_path = '/content'

# Список для хранения всех DataFrame
all_dfs = []

# Читаем все CSV файлы
print("Чтение CSV файлов...")
csv_files = [f for f in os.listdir(drive_path) if f.endswith('.csv')]
for file in csv_files:
    print(f"Чтение {file}...")
    try:
        df = pd.read_csv(os.path.join(drive_path, file), encoding='utf-8', low_memory=False)
        df['source_file'] = file
        all_dfs.append(df)
        print(f"Успешно прочитано {len(df):,} строк")
    except Exception as e:
        print(f"Ошибка при чтении {file}: {str(e)}")

# Читаем все Excel файлы
print("\nЧтение Excel файлов...")
excel_files = [f for f in os.listdir(drive_path) if f.endswith('.xlsx')]
for file in excel_files:
    print(f"Чтение {file}...")
    try:
        df = pd.read_excel(os.path.join(drive_path, file), engine='openpyxl')
        df['source_file'] = file
        all_dfs.append(df)
        print(f"Успешно прочитано {len(df):,} строк")
    except Exception as e:
        print(f"Ошибка при чтении {file}: {str(e)}")

# Читаем все архивы
print("\nЧтение архивов...")
archive_files = [f for f in os.listdir(drive_path) if f.endswith('.zip')]
for file in archive_files:
    print(f"Чтение архива {file}...")
    try:
        with zipfile.ZipFile(os.path.join(drive_path, file), 'r') as zip_ref:
            temp_dir = tempfile.mkdtemp()
            zip_ref.extractall(temp_dir)

            for root, dirs, files in os.walk(temp_dir):
                for f in files:
                    if f.endswith('.csv'):
                        df = pd.read_csv(os.path.join(root, f), low_memory=False)
                        df['source_file'] = f"{file}/{f}"
                        all_dfs.append(df)
                    elif f.endswith('.xlsx'):
                        df = pd.read_excel(os.path.join(root, f), engine='openpyxl')
                        df['source_file'] = f"{file}/{f}"
                        all_dfs.append(df)

            shutil.rmtree(temp_dir)
        print(f"Успешно прочитано {len([f for f in files if f.endswith(('.csv', '.xlsx'))])} файлов из архива")
    except Exception as e:
        print(f"Ошибка при чтении архива {file}: {str(e)}")

# Колонки с названием объекта интеллектуальной собственности, на которую оформлен патент, делаем с одинаковым названием 'object name' ===
name_columns = ['industrial design name', 'invention name', 'utility model name']

for idx, df in enumerate(all_dfs):
    for col in name_columns:
        if col in df.columns:
            df.rename(columns={col: 'object name'}, inplace=True)
    all_dfs[idx] = df


# Объединяем все данные
if all_dfs:
    # Находим общие колонки
    common_columns = set.intersection(*[set(df.columns) for df in all_dfs])

    # Объединяем только по общим колонкам
    final_df = pd.concat([df[list(common_columns)] for df in all_dfs], ignore_index=True)

    # Удалением дубликаты по 'application number' (номер заявки на патент уникальный, иначе это дублирующаяся информация)
    if 'application number' in final_df.columns:
        before_dedup = len(final_df)
        final_df = final_df.drop_duplicates(subset=['application number'], keep='first').reset_index(drop=True)
        after_dedup = len(final_df)
        removed = before_dedup - after_dedup
    else:
        print("Пропускаем проверку на дубликаты.")

    print(f"\nИтоговый размер данных: {len(final_df):,} строк x {len(final_df.columns)} колонок")

        # Выводим DataFrame
    final_df
else:
    print("Не удалось прочитать ни один файл!")

Чтение CSV файлов...
Чтение ПМОД.csv...
Успешно прочитано 222,362 строк
Чтение изобретенияОД.csv...
Успешно прочитано 813,633 строк
Чтение ПООД.csv...
Успешно прочитано 101,673 строк
Чтение ®ІЃ°а•в•≠®пОД.csv...
Успешно прочитано 813,633 строк

Чтение Excel файлов...
Чтение База по полезным моделям.xlsx...
Успешно прочитано 223,891 строк
Чтение База по промышленным образцам.xlsx...
Успешно прочитано 205,707 строк
Чтение База по изобретениям.xlsx...
Успешно прочитано 813,608 строк

Чтение архивов...

Итоговый размер данных: 1,135,794 строк x 29 колонок


In [ ]:
# Пересталвяем колонки в порядке как в исходных файлах для удобства
desired_order = [
    'registration number', 'registration date', 'application number', 'application date',
    'authors', 'authors in latin', 'patent holders', 'patent holders in latin',
    'correspondence address', 'correspondence address in latin', 'object name',
    'patent starting date', 'receipt date of additional data to application',
    'date of application to which additional data has been received',
    'number of application to which additional data has been received',
    'initial application number', 'initial application date', 'initial application priority date',
    'previous application number', 'previous application date',
    'paris convention priority number', 'paris convention priority date',
    'paris convention priority country code', 'patent grant publish date',
    'patent grant publish number', 'revoked patent number', 'expiration date', 'actual'
]

final_columns = [col for col in desired_order if col in final_df.columns]
remaining_columns = [col for col in final_df.columns if col not in final_columns]
final_df = final_df[final_columns + remaining_columns]
display(final_df)


,registration number,registration date,application number,application date,authors,authors in latin,patent holders,patent holders in latin,correspondence address,correspondence address in latin,object name,patent starting date,receipt date of additional data to application,date of application to which additional data has been received,number of application to which additional data has been received,initial application number,initial application date,initial application priority date,previous application number,previous application date,paris convention priority number,paris convention priority date,paris convention priority country code,patent grant publish date,patent grant publish number,revoked patent number,expiration date,actual,source_file
0,﻿1,19940625.0,93030652.0,19930701.0,Чалаганидзе Ш.И.\r\nКокая Г.Г.,\r\n,Напетваридзе С.О. (RU)\r\nЧалаганидзе Шота Иванович (GE),\r\n,NaN,NaN,Устройство для облесения крутых горных склонов,19930701.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19940625.0,NaN,NaN,NaN,True,ПМОД.csv
1,2,19940625.0,93015739.0,19930324.0,Дронов Владимир Дмитриевич\r\nКолобаев Владимир Никифорович\r\nПрохоров Александр Андреевич\r\nЛукьянченко Павел Петрович\r\nСтоляров Евгений Александрович\r\nГоловко Алексей Григорьевич\r\nШорников Игорь Евгеньевич\r\nСавкин Михаил Николаевич,\r\n\r\n\r\n\r\n\r\n\r\n\r\n,Дронов Владимир Дмитриевич\r\nКолобаев Владимир Никифорович\r\nПрохоров Александр Андреевич\r\nЛукьянченко Павел Петрович\r\nСтоляров Евгений Александрович\r\nГоловко Алексей Григорьевич\r\nШорников Игорь Евгеньевич\r\nСавкин Михаил Николаевич,\r\n\r\n\r\n\r\n\r\n\r\n\r\n,NaN,NaN,Регулятор вакуума,19930324.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19940625.0,NaN,NaN,NaN,True,ПМОД.csv
2,3,19940625.0,93015868.0,19930325.0,Звонов Б.М.\r\nТерсков Н.Г.\r\nВаликов А.В.,\r\n\r\n,"Малое производственное предприятие ""Пирс""",NaN,NaN,NaN,Устройство для отпугивания птиц,19930325.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19940625.0,NaN,NaN,NaN,True,ПМОД.csv
3,4,19940625.0,5061235.0,19920902.0,Селиванов Виктор Викторович,NaN,Селиванов Виктор Викторович,NaN,NaN,NaN,Устройство для тренировки аккомодационной мышцы,19920902.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19940625.0,NaN,NaN,NaN,True,ПМОД.csv
4,5,19940625.0,92008521.0,19921125.0,Васильев Михаил Григорьевич,NaN,Васильев Михаил Григорьевич,NaN,NaN,NaN,Прибор для акупунктуры,19921125.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19940625.0,NaN,NaN,NaN,True,ПМОД.csv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1135789,141705,20240425.0,2023504523.0,20231025.0,Кучин Семен Геннадьевич (RU),NaN,Кучин Семен Геннадьевич (RU),NaN,"640002,\nг. Курган, п. Увал, \nул. Трактовая, д. 1, корп. 19, кв. 172,\nКучину С.Г.",NaN,КОНВЕРТ-УПАКОВКА ДЛЯ КОМПАКТ-ДИСКОВ,20231025.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20240425.0,5.0,NaN,20281025.0,True,ПООД.csv
1135790,141706,20240425.0,2023504951.0,20231006.0,Сайгитов Махач Тимирбулатович (RU),NaN,Сайгитов Махач Тимирбулатович (RU),NaN,"196084, \nСанкт- Петербург, \nМосковский проспект, д. 75, стр. А, а/я 143,\nРассохиной Н.А.",NaN,ДИФФУЗОР ДЛЯ ПОТОЛКА,20231006.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20240425.0,5.0,NaN,20281006.0,True,ПООД.csv
1135791,141707,20240425.0,2023505528.0,20231108.0,Кучин Семен Геннадьевич (RU),NaN,Кучин Семен Геннадьевич (RU),NaN,"640002, \nг. Курган, п. Увал, \nул. Трактовая, д. 1/19, кв. 172, \nКучин С.Г.",NaN,ПРИКЛАД ДЛЯ ПИСТОЛЕТА,20231108.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20240425.0,5.0,NaN,20281108.0,True,ПООД.csv
1135792,141708,20240426.0,2023505924.0,20231128.0,Новиков Александр Владимирович (RU),NaN,"Общество с ограниченной ответственностью ""Нова-Строй"" (RU)","Obshestvo s ogranichennoi otvetstvennostiu ""Nova-Stroy"" (RU)","659309, \nАлтайский край, \nг. Бийск, \nпер. Уральский, д. 5а, \nНовиковой Г.Ю.",NaN,ПРИТОЧНОЕ ВЕНТИЛЯЦИОННОЕ УСТРОЙСТВО,20231128.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

Итоговый датафрейм содержит следующие столбцы:

registration number - официальный номер регистрации патента;

registration date - дата регистрации патента в патентном ведомстве;

application number - номер заявки на патент;

application date - дата подачи заявки;

authors - имена изобретателей на русском языке;

authors in latin - имена изобретателей латиницей;

patent holders - правообладатели (организации или физлица, владеющие правами на патент, на русском),

patent holders in latin - правообладатели (организации или физлица, латиницей);

correspondence address - адрес для связи,

correspondence address in latin - адрес для связи (латиницей),

object name - название объекта интеллектуальной собственности (например, название изобретения, полезной модели или промышленного образца);

patent starting date - дата начала действия патента

receipt date of additional data to application - дата получения дополнительных материалов к заявке (если были досланы после подачи)

date of application to which additional data has been received - дата основной заявки, к которой относились дополнительные данные

number of application to which additional data has been received - номер основной заявки, к которой относились дополнительные данные

initial application number - номер первоначальной заявки (если это продолжение или разделение заявки)

initial application date - дата первоначальной заявки

initial application priority date - дата приоритета первоначальной заявки (важно для определения новизны)

previous application number - номер предыдущей заявки (если была цепочка заявок)

previous application date - дата предыдущей заявки

paris convention priority number - номер заявки, по которой запрашивается приоритет в рамках Парижской конвенции (международный приоритет)

paris convention priority date - дата заявки по Парижской конвенции

paris convention priority country code - код страны, по которой запрашивается приоритет (например, RU, US, CN)

patent grant publish date - дата публикации выдачи патента (когда информация официально опубликована)

patent grant publish number - номер публикации о выдаче патента

revoked patent number - номер аннулированного (отозванного) патента, если есть

expiration date - дата окончания действия патента (например, по истечении срока или в случае отказа)

actual - актуальный статус (например, действует / утратил силу / аннулирован)

source_file - файл, из которого взята информация

publication URL - ссылка на информацию о публикации патента

In [ ]:
final_df1=final_df.copy()

In [ ]:

import numpy as np

def convert_dates_to_european_format(dataframe):
    processed = dataframe.copy()

    # Ищем столбцы, содержащие 'date' в названии (без учета регистра)
    date_columns = [col for col in processed.columns
                   if 'date' in col.lower()]

    # Если нет подходящих столбцов, сообщаем и возвращаем исходные данные
    if not date_columns:
        print("Столбцы с датами не обнаружены.")
        return processed

    print(f"Обнаружены столбцы с датами: {', '.join(date_columns)}")

    # Обрабатываем каждый столбец с датами
    for column in date_columns:
        print(f"\nОбработка столбца '{column}':")

        # Сохраняем оригинальные значения для сравнения
        original_values = processed[column].copy()

        try:
            # 1: Преобразуем числа в строки и удаляем .0 в конце
            cleaned_values = (
                processed[column]
                .astype(str)
                .str.replace(r'\.0$', '', regex=True)
            )

            # 2: Конвертируем в datetime с указанием формата
            date_values = pd.to_datetime(cleaned_values,
                                       format='%Y%m%d',
                                       errors='coerce')

            # 3: Форматируем
            dates = date_values.dt.strftime('%d.%m.%Y')

            # Заменяем ошибки преобразования на NaN
            dates = dates.replace('NaT', np.nan)

            # Сохраняем результат
            processed[column] = dates

            # Выводим статистику
            successful = dates.notna().sum()
            total = len(processed)
            print(f"Успешно преобразовано: {successful} из {total} записей")

            # Показываем примеры преобразований
            print("\nПримеры изменений:")
            print(pd.DataFrame({
                'Исходное значение': original_values.head(3),
                'Полученное значение': processed[column].head(3)
            }))

        except Exception as error:
            print(f"Ошибка при обработке столбца {column}: {str(error)}")
            # В случае ошибки возвращаем исходные значения
            processed[column] = original_values

    print("\nОбработка завершена. Возвращаем результат...")
    return processed

# Пример использования
print("Начинаем обработку данных...")
final_df = convert_dates_to_european_format(final_df)



Начинаем обработку данных...
Обнаружены столбцы с датами: registration date, application date, patent starting date, receipt date of additional data to application, date of application to which additional data has been received, initial application date, initial application priority date, previous application date, paris convention priority date, patent grant publish date, expiration date

Обработка столбца 'registration date':
Успешно преобразовано: 1134248 из 1135794 записей

Примеры изменений:
  Исходное значение Полученное значение
0        19940625.0          25.06.1994
1        19940625.0          25.06.1994
2        19940625.0          25.06.1994

Обработка столбца 'application date':
Успешно преобразовано: 1135793 из 1135794 записей

Примеры изменений:
  Исходное значение Полученное значение
0        19930701.0          01.07.1993
1        19930324.0          24.03.1993
2        19930325.0          25.03.1993

Обработка столбца 'patent starting date':
Успешно преобразовано: 112

In [ ]:
final_df

,registration number,registration date,application number,application date,authors,authors in latin,patent holders,patent holders in latin,correspondence address,correspondence address in latin,object name,patent starting date,receipt date of additional data to application,date of application to which additional data has been received,number of application to which additional data has been received,initial application number,initial application date,initial application priority date,previous application number,previous application date,paris convention priority number,paris convention priority date,paris convention priority country code,patent grant publish date,patent grant publish number,revoked patent number,expiration date,actual,source_file
0,﻿1,25.06.1994,93030652.0,01.07.1993,Чалаганидзе Ш.И.\r\nКокая Г.Г.,\r\n,Напетваридзе С.О. (RU)\r\nЧалаганидзе Шота Иванович (GE),\r\n,NaN,NaN,Устройство для облесения крутых горных склонов,01.07.1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.06.1994,NaN,NaN,NaN,True,ПМОД.csv
1,2,25.06.1994,93015739.0,24.03.1993,Дронов Владимир Дмитриевич\r\nКолобаев Владимир Никифорович\r\nПрохоров Александр Андреевич\r\nЛукьянченко Павел Петрович\r\nСтоляров Евгений Александрович\r\nГоловко Алексей Григорьевич\r\nШорников Игорь Евгеньевич\r\nСавкин Михаил Николаевич,\r\n\r\n\r\n\r\n\r\n\r\n\r\n,Дронов Владимир Дмитриевич\r\nКолобаев Владимир Никифорович\r\nПрохоров Александр Андреевич\r\nЛукьянченко Павел Петрович\r\nСтоляров Евгений Александрович\r\nГоловко Алексей Григорьевич\r\nШорников Игорь Евгеньевич\r\nСавкин Михаил Николаевич,\r\n\r\n\r\n\r\n\r\n\r\n\r\n,NaN,NaN,Регулятор вакуума,24.03.1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.06.1994,NaN,NaN,NaN,True,ПМОД.csv
2,3,25.06.1994,93015868.0,25.03.1993,Звонов Б.М.\r\nТерсков Н.Г.\r\nВаликов А.В.,\r\n\r\n,"Малое производственное предприятие ""Пирс""",NaN,NaN,NaN,Устройство для отпугивания птиц,25.03.1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.06.1994,NaN,NaN,NaN,True,ПМОД.csv
3,4,25.06.1994,5061235.0,02.09.1992,Селиванов Виктор Викторович,NaN,Селиванов Виктор Викторович,NaN,NaN,NaN,Устройство для тренировки аккомодационной мышцы,02.09.1992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.06.1994,NaN,NaN,NaN,True,ПМОД.csv
4,5,25.06.1994,92008521.0,25.11.1992,Васильев Михаил Григорьевич,NaN,Васильев Михаил Григорьевич,NaN,NaN,NaN,Прибор для акупунктуры,25.11.1992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.06.1994,NaN,NaN,NaN,True,ПМОД.csv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1135789,141705,25.04.2024,2023504523.0,25.10.2023,Кучин Семен Геннадьевич (RU),NaN,Кучин Семен Геннадьевич (RU),NaN,"640002,\nг. Курган, п. Увал, \nул. Трактовая, д. 1, корп. 19, кв. 172,\nКучину С.Г.",NaN,КОНВЕРТ-УПАКОВКА ДЛЯ КОМПАКТ-ДИСКОВ,25.10.2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.04.2024,5.0,NaN,25.10.2028,True,ПООД.csv
1135790,141706,25.04.2024,2023504951.0,06.10.2023,Сайгитов Махач Тимирбулатович (RU),NaN,Сайгитов Махач Тимирбулатович (RU),NaN,"196084, \nСанкт- Петербург, \nМосковский проспект, д. 75, стр. А, а/я 143,\nРассохиной Н.А.",NaN,ДИФФУЗОР ДЛЯ ПОТОЛКА,06.10.2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.04.2024,5.0,NaN,06.10.2028,True,ПООД.csv
1135791,141707,25.04.2024,2023505528.0,08.11.2023,Кучин Семен Геннадьевич (RU),NaN,Кучин Семен Геннадьевич (RU),NaN,"640002, \nг. Курган, п. Увал, \nул. Трактовая, д. 1/19, кв. 172, \nКучин С.Г.",NaN,ПРИКЛАД ДЛЯ ПИСТОЛЕТА,08.11.2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.04.2024,5.0,NaN,08.11.2028,True,ПООД.csv
1135792,141708,26.04.2024,2023505924.0,28.11.2023,Новиков Александр Владимирович (RU),NaN,"Общество с ограниченной ответственностью ""Нова-Строй"" (RU)","Obshestvo s ogranichennoi otvetstvennostiu ""Nova-Stroy"" (RU)","659309, \nАлтайский край, \nг. Бийск, \nпер. Уральский, д. 5а, \nНовиковой Г.Ю.",NaN,ПРИТОЧНОЕ ВЕНТИЛЯЦИОННОЕ УСТРОЙСТВО,28.11.2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [ ]:
import re
from collections import defaultdict
mpk_sections = {
    'A': {
        'name': 'Удовлетворение жизненных потребностей человека',
        'keywords': ['медицин', 'хирург', 'диагност', 'терапи', 'аппарат медицин',
                    'пищ', 'продукт', 'питани', 'кулинар', 'табак', 'мебель',
                    'предмет домашн', 'бытов', 'санитар']
    },
    'B': {
        'name': 'Различные технологические процессы; транспортирование',
        'keywords': ['обработк', 'транспорт', 'перевозк', 'конвейер', 'упаковк',
                    'хранен', 'подъемн', 'погрузочн', 'сортировк', 'процесс технологич']
    },
    'C': {
        'name': 'Химия; металлургия',
        'keywords': ['химическ', 'металлург', 'сплав', 'композит', 'полимер',
                    'удобрен', 'нефт', 'газ', 'кокс', 'крас', 'лак', 'волокн']
    },
    'D': {
        'name': 'Текстиль; бумага',
        'keywords': ['текстил', 'ткань', 'волокн', 'пряж', 'бумаг', 'картон',
                    'издел бумажн', 'печат', 'переплетен']
    },
    'E': {
        'name': 'Строительство и горное дело',
        'keywords': ['строительств', 'здан', 'сооружен', 'дорог', 'мост', 'тоннел',
                    'горн', 'шахт', 'бурен', 'землян', 'фундамент', 'крыш']
    },
    'F': {
        'name': 'Машиностроение; освещение; отопление; двигатели и насосы; оружие и боеприпасы; взрывные работы',
        'keywords': ['машин', 'двигател', 'насос', 'турбин', 'оружи', 'боеприпас',
                    'взрыв', 'освещен', 'отоплен', 'охлажден', 'вентиляц', 'двигатель внутренн']
    },
    'G': {
        'name': 'Физика',
        'keywords': ['физическ', 'оптическ', 'измерен', 'тестирован', 'прибор',
                    'регистрац', 'акустическ', 'термодинамик', 'ядерн', 'радио', 'спектроскоп']
    },
    'H': {
        'name': 'Электричество',
        'keywords': ['электрическ', 'электрон', 'схем', 'микропроцессор', 'полупроводник',
                    'батаре', 'аккумулятор', 'трансформатор', 'генератор', 'реле', 'передатчик']
    }
}

def find_mpk_section(patent_title):
    """Определяет подходящий раздел МПК для названия патента"""
    if pd.isna(patent_title):
        return None

    # Приводим к нижнему регистру для удобства сравнения
    title = str(patent_title).lower()
    matches = defaultdict(int)

    # Проверяем каждое ключевое слово из всех разделов
    for section_code, section_data in mpk_sections.items():
        for keyword in section_data['keywords']:
            if re.search(rf'\b{keyword}', title):
                matches[section_code] += 1

    # Возвращаем раздел с наибольшим количеством совпадений
    if matches:
        best_match = max(matches.items(), key=lambda x: x[1])
        return best_match[0]
    return None

def add_mpk_to_data(df):
    """Основная функция - добавляет классификацию МПК в DataFrame"""

    # Проверим, что у нас есть нужные данные
    if not isinstance(df, pd.DataFrame):
        print('Ошибка: нужно передать pandas DataFrame')
        return df

    if 'object name' not in df.columns:
        print('Ошибка: в данных отсутствует колонка с названиями патентов')
        return df

    print('Начинаю анализ патентов...')

    # Применяем нашу функцию классификации
    df['mpk_sections'] = df['object name'].apply(find_mpk_section)

    # Добавляем человеко-читаемые названия разделов
    df['mpk_section_name'] = df['mpk_sections'].map(
        lambda x: mpk_sections.get(x, {}).get('name', 'Не определено')
    )

    # Посчитаем статистику
    total_patents = len(df)
    classified = df['mpk_sections'].notna().sum()

    print('\nРезультаты:')
    print(f'Всего обработано патентов: {total_patents}')
    print(f'Успешно классифицировано: {classified} ({classified/total_patents:.1%})')
    print(f'Не удалось определить раздел: {total_patents - classified}')

    if classified > 0:
        print('\nРаспределение по разделам:')
        print(df['mpk_section_name'].value_counts().to_string())

    return df

# Применяем к нашим данным
if 'final_df' in globals():
    print('Обнаружен DataFrame final_df, начинаю обработку...')
    final_df = add_mpk_to_data(final_df)
    print('Новые колонки: mpk_section (код) и mpk_section_name (название)')
else:
    print('Ошибка: не найден DataFrame finale_df для обработки')

Обнаружен DataFrame final_df, начинаю обработку...
Начинаю анализ патентов...

Результаты:
Всего обработано патентов: 1135794
Успешно классифицировано: 311717 (27.4%)
Не удалось определить раздел: 824077

Распределение по разделам:
mpk_section_name
Не определено                                                                                     824077
Химия; металлургия                                                                                 61017
Различные технологические процессы; транспортирование                                              60603
Машиностроение; освещение; отопление; двигатели и насосы; оружие и боеприпасы; взрывные работы     51791
Удовлетворение жизненных потребностей человека                                                     46408
Физика                                                                                             38015
Электричество                                                                                      28324
Строительство и 

In [ ]:
final_df

,registration number,registration date,application number,application date,authors,authors in latin,patent holders,patent holders in latin,correspondence address,correspondence address in latin,object name,patent starting date,receipt date of additional data to application,date of application to which additional data has been received,number of application to which additional data has been received,initial application number,initial application date,initial application priority date,previous application number,previous application date,paris convention priority number,paris convention priority date,paris convention priority country code,patent grant publish date,patent grant publish number,revoked patent number,expiration date,actual,source_file,mpk_sections,mpk_section_name
0,﻿1,25.06.1994,93030652.0,01.07.1993,Чалаганидзе Ш.И.\r\nКокая Г.Г.,\r\n,Напетваридзе С.О. (RU)\r\nЧалаганидзе Шота Иванович (GE),\r\n,NaN,NaN,Устройство для облесения крутых горных склонов,01.07.1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.06.1994,NaN,NaN,NaN,True,ПМОД.csv,E,Строительство и горное дело
1,2,25.06.1994,93015739.0,24.03.1993,Дронов Владимир Дмитриевич\r\nКолобаев Владимир Никифорович\r\nПрохоров Александр Андреевич\r\nЛукьянченко Павел Петрович\r\nСтоляров Евгений Александрович\r\nГоловко Алексей Григорьевич\r\nШорников Игорь Евгеньевич\r\nСавкин Михаил Николаевич,\r\n\r\n\r\n\r\n\r\n\r\n\r\n,Дронов Владимир Дмитриевич\r\nКолобаев Владимир Никифорович\r\nПрохоров Александр Андреевич\r\nЛукьянченко Павел Петрович\r\nСтоляров Евгений Александрович\r\nГоловко Алексей Григорьевич\r\nШорников Игорь Евгеньевич\r\nСавкин Михаил Николаевич,\r\n\r\n\r\n\r\n\r\n\r\n\r\n,NaN,NaN,Регулятор вакуума,24.03.1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.06.1994,NaN,NaN,NaN,True,ПМОД.csv,None,Не определено
2,3,25.06.1994,93015868.0,25.03.1993,Звонов Б.М.\r\nТерсков Н.Г.\r\nВаликов А.В.,\r\n\r\n,"Малое производственное предприятие ""Пирс""",NaN,NaN,NaN,Устройство для отпугивания птиц,25.03.1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.06.1994,NaN,NaN,NaN,True,ПМОД.csv,None,Не определено
3,4,25.06.1994,5061235.0,02.09.1992,Селиванов Виктор Викторович,NaN,Селиванов Виктор Викторович,NaN,NaN,NaN,Устройство для тренировки аккомодационной мышцы,02.09.1992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.06.1994,NaN,NaN,NaN,True,ПМОД.csv,None,Не определено
4,5,25.06.1994,92008521.0,25.11.1992,Васильев Михаил Григорьевич,NaN,Васильев Михаил Григорьевич,NaN,NaN,NaN,Прибор для акупунктуры,25.11.1992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.06.1994,NaN,NaN,NaN,True,ПМОД.csv,G,Физика
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1135789,141705,25.04.2024,2023504523.0,25.10.2023,Кучин Семен Геннадьевич (RU),NaN,Кучин Семен Геннадьевич (RU),NaN,"640002,\nг. Курган, п. Увал, \nул. Трактовая, д. 1, корп. 19, кв. 172,\nКучину С.Г.",NaN,КОНВЕРТ-УПАКОВКА ДЛЯ КОМПАКТ-ДИСКОВ,25.10.2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.04.2024,5.0,NaN,25.10.2028,True,ПООД.csv,B,Различные технологические процессы; транспортирование
1135790,141706,25.04.2024,2023504951.0,06.10.2023,Сайгитов Махач Тимирбулатович (RU),NaN,Сайгитов Махач Тимирбулатович (RU),NaN,"196084, \nСанкт- Петербург, \nМосковский проспект, д. 75, стр. А, а/я 143,\nРассохиной Н.А.",NaN,ДИФФУЗОР ДЛЯ ПОТОЛКА,06.10.2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.04.2024,5.0,NaN,06.10.2028,True,ПООД.csv,None,Не определено
1135791,141707,25.04.2024,2023505528.0,08.11.2023,Кучин Семен Геннадьевич (RU),NaN,Кучин Семен Геннадьевич (RU),NaN,"640002, \nг. Курган, п. Увал, \nул. Трактовая, д. 1/19, кв. 172, \nКучин С.Г.",NaN,ПРИКЛАД ДЛЯ ПИСТОЛЕТА,08.11.2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.04.2024,5.0,NaN,08.11.2028,True,ПООД.csv,None,Не определено
1135792,141708,26.04.2024,2023505924.0,28.11.2023,Новиков Александр Владимирович (RU),NaN,"Общество с ограниченной ответственностью ""Нова-Строй"" (RU)","Obshes

In [ ]:
# Перейдем к очистке и стандартизации наименований организаций
# Словарь для замены аббревиатур на полные названия
abbreviation_map = {
    r'\bАО\b': 'Акционерное общество',
    r'\bОАО\b': 'Открытое акционерное общество',
    r'\bЗАО\b': 'Закрытое акционерное общество',
    r'\bООО\b': 'Общество с ограниченной ответственностью',
    r'\bПАО\b': 'Публичное акционерное общество',
    r'\bАНО\b': 'Автономная некоммерческая организация',
    r'\bГУ\b': 'Государственное учреждение',
    r'\bФГУ\b': 'Федеральное государственное учреждение',
    r'\bНИИ\b': 'Научно-исследовательский институт',
    r'\bВУЗ\b': 'Высшее учебное заведение'
}

# Слова, характерные для организаций (чтобы отличить от ФИО)
organization_keywords = [
    'общество', 'учреждение', 'институт', 'университет', 'организация', 'фонд', 'компания'
]

def is_fio(text):
    """ Проверяем, похоже ли на ФИО (3 слова кириллицей) """
    words = text.strip().split()
    if len(words) == 3 and all(re.match(r'^[а-я\-]+$', w, re.IGNORECASE) for w in words):
        return True
    return False

def clean_patent_holder(name):
    if pd.isna(name):
        return 'Не указано'

    cleaned = name.strip()

    # Удаляем (RU), (DE), (CN) и т.п.
    cleaned = re.sub(r'\s*\([A-Z]{2}\)\s*', '', cleaned)

    # Убираем \r, \n, \\r, \\n
    cleaned = re.sub(r'\\r|\\n|\r|\n', ' ', cleaned)

    # Приводим кавычки к одному виду
    cleaned = cleaned.replace('«', '"').replace('»', '"').replace("'", '"')

    # Приводим всё в нижний регистр
    cleaned = cleaned.lower()

    # Заменяем аббревиатуры на полные названия
    for pattern, full in abbreviation_map.items():
        cleaned = re.sub(pattern, full, cleaned, flags=re.IGNORECASE)

    # Сокращаем лишние пробелы
    cleaned = re.sub(r'\s+', ' ', cleaned).strip()

    # Обработка ФИО или организации
    if is_fio(cleaned):
        # Каждое слово с большой буквы
        cleaned = cleaned.title()
    else:
        # Только первое слово + первое после кавычек с большой буквы
        cleaned = cleaned[0].upper() + cleaned[1:] if cleaned else ''
        cleaned = re.sub(r'(")(\s*)(\w)', lambda m: m.group(1) + m.group(2) + m.group(3).upper(), cleaned)

    return cleaned

def standardize_patent_holders(df):
    processed = df.copy()

    if 'patent holders' not in processed.columns:
        print("Ошибка: нет колонки 'patent holders'")
        return processed

    if 'patent holders in latin' in processed.columns:
        processed = processed.drop(columns=['patent holders in latin'])
        print("Удалена колонка 'patent holders in latin' (практически всегда пустая)")

    print("\nПримеры до очистки:")
    for val in processed['patent holders'].head(5):
        print(f" - {val}")

    processed['patent holders'] = processed['patent holders'].apply(clean_patent_holder)

    print("\nПримеры после очистки:")
    for val in processed['patent holders'].head(5):
        print(f" - {val}")

    print("\nОчистка завершена. Возвращаем обновлённый DataFrame.")
    return processed

# Применяем к нашим данным
if 'final_df' in globals():
    print("Обнаружен DataFrame final_df, начинаю обработку...")
    final_df = standardize_patent_holders(final_df)
else:
    print("Ошибка: DataFrame final_df не найден.")



Обнаружен DataFrame final_df, начинаю обработку...
Удалена колонка 'patent holders in latin' (практически всегда пустая)

Примеры до очистки:
 - Напетваридзе С.О. (RU)
Чалаганидзе Шота Иванович (GE)
 - Дронов Владимир Дмитриевич
Колобаев Владимир Никифорович
Прохоров Александр Андреевич
Лукьянченко Павел Петрович
Столяров Евгений Александрович
Головко Алексей Григорьевич
Шорников Игорь Евгеньевич
Савкин Михаил Николаевич
 - Малое производственное предприятие "Пирс"
 - Селиванов Виктор Викторович
 - Васильев Михаил Григорьевич

Примеры после очистки:
 - Напетваридзе с.о.чалаганидзе шота иванович
 - Дронов владимир дмитриевич колобаев владимир никифорович прохоров александр андреевич лукьянченко павел петрович столяров евгений александрович головко алексей григорьевич шорников игорь евгеньевич савкин михаил николаевич
 - Малое производственное предприятие "Пирс"
 - Селиванов Виктор Викторович
 - Васильев Михаил Григорьевич

Очистка завершена. Возвращаем обновлённый DataFrame.


In [ ]:
final_df

,registration number,registration date,application number,application date,authors,authors in latin,patent holders,correspondence address,correspondence address in latin,object name,patent starting date,receipt date of additional data to application,date of application to which additional data has been received,number of application to which additional data has been received,initial application number,initial application date,initial application priority date,previous application number,previous application date,paris convention priority number,paris convention priority date,paris convention priority country code,patent grant publish date,patent grant publish number,revoked patent number,expiration date,actual,source_file,mpk_sections,mpk_section_name
0,﻿1,25.06.1994,93030652.0,01.07.1993,Чалаганидзе Ш.И.\r\nКокая Г.Г.,\r\n,Напетваридзе с.о.чалаганидзе шота иванович,NaN,NaN,Устройство для облесения крутых горных склонов,01.07.1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.06.1994,NaN,NaN,NaN,True,ПМОД.csv,E,Строительство и горное дело
1,2,25.06.1994,93015739.0,24.03.1993,Дронов Владимир Дмитриевич\r\nКолобаев Владимир Никифорович\r\nПрохоров Александр Андреевич\r\nЛукьянченко Павел Петрович\r\nСтоляров Евгений Александрович\r\nГоловко Алексей Григорьевич\r\nШорников Игорь Евгеньевич\r\nСавкин Михаил Николаевич,\r\n\r\n\r\n\r\n\r\n\r\n\r\n,Дронов владимир дмитриевич колобаев владимир никифорович прохоров александр андреевич лукьянченко павел петрович столяров евгений александрович головко алексей григорьевич шорников игорь евгеньевич савкин михаил николаевич,NaN,NaN,Регулятор вакуума,24.03.1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.06.1994,NaN,NaN,NaN,True,ПМОД.csv,None,Не определено
2,3,25.06.1994,93015868.0,25.03.1993,Звонов Б.М.\r\nТерсков Н.Г.\r\nВаликов А.В.,\r\n\r\n,"Малое производственное предприятие ""Пирс""",NaN,NaN,Устройство для отпугивания птиц,25.03.1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.06.1994,NaN,NaN,NaN,True,ПМОД.csv,None,Не определено
3,4,25.06.1994,5061235.0,02.09.1992,Селиванов Виктор Викторович,NaN,Селиванов Виктор Викторович,NaN,NaN,Устройство для тренировки аккомодационной мышцы,02.09.1992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.06.1994,NaN,NaN,NaN,True,ПМОД.csv,None,Не определено
4,5,25.06.1994,92008521.0,25.11.1992,Васильев Михаил Григорьевич,NaN,Васильев Михаил Григорьевич,NaN,NaN,Прибор для акупунктуры,25.11.1992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.06.1994,NaN,NaN,NaN,True,ПМОД.csv,G,Физика
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1135789,141705,25.04.2024,2023504523.0,25.10.2023,Кучин Семен Геннадьевич (RU),NaN,Кучин Семен Геннадьевич,"640002,\nг. Курган, п. Увал, \nул. Трактовая, д. 1, корп. 19, кв. 172,\nКучину С.Г.",NaN,КОНВЕРТ-УПАКОВКА ДЛЯ КОМПАКТ-ДИСКОВ,25.10.2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.04.2024,5.0,NaN,25.10.2028,True,ПООД.csv,B,Различные технологические процессы; транспортирование
1135790,141706,25.04.2024,2023504951.0,06.10.2023,Сайгитов Махач Тимирбулатович (RU),NaN,Сайгитов Махач Тимирбулатович,"196084, \nСанкт- Петербург, \nМосковский проспект, д. 75, стр. А, а/я 143,\nРассохиной Н.А.",NaN,ДИФФУЗОР ДЛЯ ПОТОЛКА,06.10.2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.04.2024,5.0,NaN,06.10.2028,True,ПООД.csv,None,Не определено
1135791,141707,25.04.2024,2023505528.0,08.11.2023,Кучин Семен Геннадьевич (RU),NaN,Кучин Семен Геннадьевич,"640002, \nг. Курган, п. Увал, \nул. Трактовая, д. 1/19, кв. 172, \nКучин С.Г.",NaN,ПРИКЛАД ДЛЯ ПИСТОЛЕТА,08.11.2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.04.2024,5.0,NaN,08.11.2028,True,ПООД.csv,None,Не определено
1135792,141708,26.04.2024,2023505924.0,28.11.2023,Новиков Александр Владимирович (RU),NaN,"Общество с ограниченной ответственностью ""Нова-строй""","659309, \nАлтайский край, \nг. Бийск, \nпер. Уральский, д. 5а, \nНовиковой Г.Ю.",NaN,ПРИТОЧНОЕ ВЕНТИЛЯЦИОННОЕ УСТРОЙСТВО,28.11.2023,NaN,NaN,NaN,NaN

In [ ]:
# Очистим от мусора остальные столбцы датафрейма
def clean_general_value(value):
    if pd.isna(value):
        return value

    # Если строка, чистим мусор
    if isinstance(value, str):
        cleaned = value.strip()

        # Убираем (RU), (DE), (CN) и подобные маркеры
        cleaned = re.sub(r'\s*\([A-Z]{2}\)\s*', '', cleaned)

        # Убираем \r, \n, \t и лишние пробелы
        cleaned = re.sub(r'\\r|\\n|\r|\n|\t', ' ', cleaned)
        cleaned = re.sub(r'\s+', ' ', cleaned).strip()

        return cleaned

    # Для других типов данных возвращаем как есть
    return value

def clean_dataframe(df):
    processed = df.copy()
    exclude_cols = ['patent holders', 'mpk_sections', 'mpk_section_name']

    for col in processed.columns:
        if col in exclude_cols:
            continue

        processed[col] = processed[col].apply(clean_general_value)

    return processed

# Применяем к нашим данным
if 'final_df' in globals():
    print("Обнаружен DataFrame final_df, начинаю обработку...")
    print("Начинаю очистку всех оставщихся колонок")
    cleaned_df = clean_dataframe(final_df)
    print("\nОчистка завершена.")
else:
    print("Ошибка: DataFrame final_df не найден.")


Обнаружен DataFrame final_df, начинаю обработку...
Начинаю очистку всех оставщихся колонок

Очистка завершена.


In [ ]:
cleaned_df

,registration number,registration date,application number,application date,authors,authors in latin,patent holders,correspondence address,correspondence address in latin,object name,patent starting date,receipt date of additional data to application,date of application to which additional data has been received,number of application to which additional data has been received,initial application number,initial application date,initial application priority date,previous application number,previous application date,paris convention priority number,paris convention priority date,paris convention priority country code,patent grant publish date,patent grant publish number,revoked patent number,expiration date,actual,source_file,mpk_sections,mpk_section_name
0,﻿1,25.06.1994,93030652.0,01.07.1993,Чалаганидзе Ш.И. Кокая Г.Г.,,Напетваридзе с.о.чалаганидзе шота иванович,NaN,NaN,Устройство для облесения крутых горных склонов,01.07.1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.06.1994,NaN,NaN,NaN,True,ПМОД.csv,E,Строительство и горное дело
1,2,25.06.1994,93015739.0,24.03.1993,Дронов Владимир Дмитриевич Колобаев Владимир Никифорович Прохоров Александр Андреевич Лукьянченко Павел Петрович Столяров Евгений Александрович Головко Алексей Григорьевич Шорников Игорь Евгеньевич Савкин Михаил Николаевич,,Дронов владимир дмитриевич колобаев владимир никифорович прохоров александр андреевич лукьянченко павел петрович столяров евгений александрович головко алексей григорьевич шорников игорь евгеньевич савкин михаил николаевич,NaN,NaN,Регулятор вакуума,24.03.1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.06.1994,NaN,NaN,NaN,True,ПМОД.csv,None,Не определено
2,3,25.06.1994,93015868.0,25.03.1993,Звонов Б.М. Терсков Н.Г. Валиков А.В.,,"Малое производственное предприятие ""Пирс""",NaN,NaN,Устройство для отпугивания птиц,25.03.1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.06.1994,NaN,NaN,NaN,True,ПМОД.csv,None,Не определено
3,4,25.06.1994,5061235.0,02.09.1992,Селиванов Виктор Викторович,NaN,Селиванов Виктор Викторович,NaN,NaN,Устройство для тренировки аккомодационной мышцы,02.09.1992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.06.1994,NaN,NaN,NaN,True,ПМОД.csv,None,Не определено
4,5,25.06.1994,92008521.0,25.11.1992,Васильев Михаил Григорьевич,NaN,Васильев Михаил Григорьевич,NaN,NaN,Прибор для акупунктуры,25.11.1992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.06.1994,NaN,NaN,NaN,True,ПМОД.csv,G,Физика
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1135789,141705,25.04.2024,2023504523.0,25.10.2023,Кучин Семен Геннадьевич,NaN,Кучин Семен Геннадьевич,"640002, г. Курган, п. Увал, ул. Трактовая, д. 1, корп. 19, кв. 172, Кучину С.Г.",NaN,КОНВЕРТ-УПАКОВКА ДЛЯ КОМПАКТ-ДИСКОВ,25.10.2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.04.2024,5.0,NaN,25.10.2028,True,ПООД.csv,B,Различные технологические процессы; транспортирование
1135790,141706,25.04.2024,2023504951.0,06.10.2023,Сайгитов Махач Тимирбулатович,NaN,Сайгитов Махач Тимирбулатович,"196084, Санкт- Петербург, Московский проспект, д. 75, стр. А, а/я 143, Рассохиной Н.А.",NaN,ДИФФУЗОР ДЛЯ ПОТОЛКА,06.10.2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.04.2024,5.0,NaN,06.10.2028,True,ПООД.csv,None,Не определено
1135791,141707,25.04.2024,2023505528.0,08.11.2023,Кучин Семен Геннадьевич,NaN,Кучин Семен Геннадьевич,"640002, г. Курган, п. Увал, ул. Трактовая, д. 1/19, кв. 172, Кучин С.Г.",NaN,ПРИКЛАД ДЛЯ ПИСТОЛЕТА,08.11.2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.04.2024,5.0,NaN,08.11.2028,True,ПООД.csv,None,Не определено
1135792,141708,26.04.2024,2023505924.0,28.11.2023,Новиков Александр Владимирович,NaN,"Общество с ограниченной ответственностью ""Нова-строй""","659309, Алтайский край, г. Бийск, пер. Уральский, д. 5а, Новиковой Г.Ю.",NaN,ПРИТОЧНОЕ ВЕНТИЛЯЦИОННОЕ УСТРОЙСТВО,28.11.2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.04.2024,5.0,NaN,28.11.2028,True,ПООД.csv,F,Машиностроение; освещение; отопление; 

In [ ]:
from collections import Counter
class PatentClassifier:
    def __init__(self):
        # Полная иерархия МПК с подклассами и группами
        self.ipc_hierarchy = {
    'A': {
        'name': 'Удовлетворение жизненных потребностей',
        'subclasses': {
            'A01': {'name': 'Сельское хозяйство; лесное хозяйство; животноводство',
                   'keywords': ['сельск', 'агро', 'ферм', 'почв', 'растен', 'удобрен', 'лесн', 'животновод']},
            'A21': {'name': 'Хлебопечение; мукомольное производство',
                   'keywords': ['хлеб', 'пекар', 'мук', 'тест', 'кондитерск']},
            'A22': {'name': 'Убой скота; переработка мяса',
                   'keywords': ['убой', 'мяс', 'скот', 'колбас', 'копчен']},
            'A23': {'name': 'Пищевые продукты; их обработка',
                   'keywords': ['пищ', 'продукт', 'питани', 'консерв', 'молочн', 'мясн', 'напиток']},
            'A24': {'name': 'Табачные изделия',
                   'keywords': ['табак', 'сигар', 'сигарет', 'курен']},
            'A41': {'name': 'Одежда',
                   'keywords': ['одежд', 'костюм', 'бель', 'галстук', 'перчат']},
            'A42': {'name': 'Головные уборы',
                   'keywords': ['шляп', 'головн', 'каск', 'кепк']},
            'A43': {'name': 'Обувь',
                   'keywords': ['обув', 'сапог', 'туфл', 'подошв']},
            'A44': {'name': 'Галантерейные изделия',
                   'keywords': ['галантере', 'пугов', 'запон', 'зонт']},
            'A45': {'name': 'Предметы личного обихода',
                   'keywords': ['чемодан', 'щетк', 'расчес', 'бритв']},
            'A46': {'name': 'Щетки',
                   'keywords': ['щетк', 'метел', 'чист']},
            'A47': {'name': 'Мебель; предметы домашнего обихода',
                   'keywords': ['мебел', 'стол', 'стул', 'кроват', 'диван', 'шкаф']},
            'A61': {'name': 'Медицина; ветеринария; гигиена',
                   'keywords': ['медицин', 'хирург', 'диагност', 'терапи', 'аппарат медицин', 'стоматолог']},
            'A62': {'name': 'Спасательные средства; противопожарная защита',
                   'keywords': ['пожар', 'спасен', 'противогаз', 'безопасност']},
            'A63': {'name': 'Спорт; игры; развлечения',
                   'keywords': ['спорт', 'игр', 'развлечен', 'музык', 'тренажер']}
        }
    },
    'B': {
        'name': 'Технологические процессы; транспортирование',
        'subclasses': {
            'B01': {'name': 'Физические или химические процессы или аппаратура вообще',
                   'keywords': ['физико-химич', 'реакц', 'катализ', 'сепарац', 'фильтр']},
            'B02': {'name': 'Дробление, измельчение или диспергирование',
                   'keywords': ['дроблен', 'измельчен', 'мельниц', 'дисперг']},
            'B03': {'name': 'Разделение материалов',
                   'keywords': ['разделен', 'сортиров', 'магнитн', 'флотац']},
            'B04': {'name': 'Центрифуги',
                   'keywords': ['центрифуг', 'сепаратор']},
            'B05': {'name': 'Распыление или нанесение покрытий',
                   'keywords': ['распылен', 'покрыт', 'крас', 'лак']},
            'B06': {'name': 'Генерация механических колебаний',
                   'keywords': ['вибрац', 'акустик', 'ультразвук']},
            'B07': {'name': 'Разделение твердых материалов',
                   'keywords': ['грохот', 'сортиров', 'сепарац']},
            'B08': {'name': 'Очистка',
                   'keywords': ['очистк', 'уборк', 'мойк']},
            'B09': {'name': 'Удаление отходов',
                   'keywords': ['отход', 'утилиз', 'переработк']},
            'B21': {'name': 'Механическая обработка металлов',
                   'keywords': ['ковк', 'штампов', 'прокат']},
            'B22': {'name': 'Литейное производство',
                   'keywords': ['лит', 'формов', 'лить']},
            'B23': {'name': 'Обработка металлов резанием',
                   'keywords': ['токар', 'фрезер', 'сверл', 'шлифов']},
            'B24': {'name': 'Шлифование, полирование',
                   'keywords': ['шлифов', 'полиров', 'абразив']},
            'B25': {'name': 'Ручные инструменты',
                   'keywords': ['инструмент', 'молоток', 'отверт', 'плоскогубц']},
            'B26': {'name': 'Ручные режущие инструменты',
                   'keywords': ['нож', 'резак', 'ножниц']},
            'B27': {'name': 'Обработка древесины',
                   'keywords': ['древ', 'пил', 'строган']},
            'B28': {'name': 'Обработка цемента, глины, камня',
                   'keywords': ['бетон', 'цемент', 'керамик', 'камен']},
            'B29': {'name': 'Обработка пластмасс',
                   'keywords': ['пластмасс', 'полимер', 'лить']},
            'B30': {'name': 'Прессы',
                   'keywords': ['пресс', 'штампов']},
            'B31': {'name': 'Изготовление бумаги',
                   'keywords': ['бумаг', 'картон', 'упаковк']},
            'B32': {'name': 'Слоистые материалы',
                   'keywords': ['слоист', 'ламинат', 'композит']},
            'B41': {'name': 'Печатное дело',
                   'keywords': ['печат', 'типограф', 'офсет']},
            'B42': {'name': 'Переплетное дело',
                   'keywords': ['книг', 'переплет', 'блокнот']},
            'B43': {'name': 'Письменные принадлежности',
                   'keywords': ['ручк', 'карандаш', 'чернил']},
            'B44': {'name': 'Декоративное искусство',
                   'keywords': ['декор', 'худож', 'мозаик']},
            'B60': {'name': 'Транспортные средства вообще',
                   'keywords': ['автомоб', 'транспорт', 'кузов', 'колес']},
            'B61': {'name': 'Железнодорожный транспорт',
                   'keywords': ['железнодорож', 'вагон', 'рельс']},
            'B62': {'name': 'Наземный транспорт',
                   'keywords': ['велосипед', 'мотоцикл', 'тележк']},
            'B63': {'name': 'Судостроение',
                   'keywords': ['суд', 'кораб', 'лодк']},
            'B64': {'name': 'Авиация, космонавтика',
                   'keywords': ['самолет', 'вертолет', 'космос']},
            'B65': {'name': 'Транспортировка, упаковка, хранение',
                   'keywords': ['конвейер', 'упаковк', 'погрузк']},
            'B66': {'name': 'Подъемные устройства',
                   'keywords': ['кран', 'лифт', 'подъемник']},
            'B67': {'name': 'Разливка жидкостей',
                   'keywords': ['разлив', 'бенз', 'налив']},
            'B68': {'name': 'Сбруя, обивка',
                   'keywords': ['седл', 'упряж', 'обивк']}
        }
    },
    'C': {
        'name': 'Химия; металлургия',
        'subclasses': {
            'C01': {'name': 'Неорганическая химия',
                   'keywords': ['неорган', 'кислот', 'соль', 'оксид']},
            'C02': {'name': 'Обработка воды',
                   'keywords': ['вод', 'очистк', 'фильтрац']},
            'C03': {'name': 'Стекло; минеральная вата',
                   'keywords': ['стекл', 'минерал', 'вата']},
            'C04': {'name': 'Цементы; бетон; керамика',
                   'keywords': ['цемент', 'бетон', 'керамик']},
            'C05': {'name': 'Удобрения; их производство',
                   'keywords': ['удобрен', 'азот', 'фосфат']},
            'C06': {'name': 'Взрывчатые вещества',
                   'keywords': ['взрывчат', 'порох', 'пиротехн']},
            'C07': {'name': 'Органическая химия',
                   'keywords': ['орган', 'углеводород', 'спирт']},
            'C08': {'name': 'Органические высокомолекулярные соединения',
                   'keywords': ['полимер', 'пластмасс', 'каучук']},
            'C09': {'name': 'Красители; полировка',
                   'keywords': ['крас', 'пигмент', 'лак']},
            'C10': {'name': 'Нефтяная, газовая промышленность',
                   'keywords': ['нефт', 'газ', 'бензин', 'масл']},
            'C11': {'name': 'Животные и растительные масла',
                   'keywords': ['масл', 'жир', 'мыл']},
            'C12': {'name': 'Биохимия; пиво; спирт',
                   'keywords': ['биохим', 'пив', 'спирт']},
            'C13': {'name': 'Сахарная промышленность',
                   'keywords': ['сахар', 'патока', 'крахмал']},
            'C14': {'name': 'Кожевенное производство',
                   'keywords': ['кож', 'дублен', 'мех']},
            'C21': {'name': 'Металлургия железа',
                   'keywords': ['чугун', 'стал', 'металлург']},
            'C22': {'name': 'Металлургия цветных металлов',
                   'keywords': ['алюмин', 'мед', 'цинк']},
            'C23': {'name': 'Покрытие металлов',
                   'keywords': ['гальван', 'покрыт', 'анодирован']},
            'C25': {'name': 'Электролитические процессы',
                   'keywords': ['электролиз', 'гальван', 'анодирован']},
            'C30': {'name': 'Кристаллография',
                   'keywords': ['кристал', 'синтез']}
        }
    },
    'D': {
        'name': 'Текстиль; бумага',
        'subclasses': {
            'D01': {'name': 'Природные или искусственные нити',
                   'keywords': ['нит', 'волокн', 'пряж']},
            'D02': {'name': 'Прядение',
                   'keywords': ['пряд', 'кручен', 'ровниц']},
            'D03': {'name': 'Ткачество',
                   'keywords': ['ткац', 'станок', 'переплетен']},
            'D04': {'name': 'Плетение',
                   'keywords': ['плетен', 'кружев', 'сетк']},
            'D05': {'name': 'Швейное производство',
                   'keywords': ['швейн', 'вышив', 'вязан']},
            'D06': {'name': 'Обработка текстиля',
                   'keywords': ['отделк', 'крашен', 'белен']},
            'D07': {'name': 'Канаты, веревки',
                   'keywords': ['канат', 'верев', 'трос']},
            'D21': {'name': 'Производство бумаги',
                   'keywords': ['бумаг', 'целлюлоз', 'картон']}
        }
    },
    'E': {
        'name': 'Строительство; горное дело',
        'subclasses': {
            'E01': {'name': 'Дорожное строительство',
                   'keywords': ['дорог', 'асфальт', 'тротуар']},
            'E02': {'name': 'Гидротехнические сооружения',
                   'keywords': ['плотин', 'канал', 'набережн']},
            'E03': {'name': 'Водоснабжение; канализация',
                   'keywords': ['водопровод', 'канализац', 'колодец']},
            'E04': {'name': 'Строительство',
                   'keywords': ['здан', 'строительств', 'фундамент']},
            'E05': {'name': 'Замки; двери',
                   'keywords': ['замок', 'двер', 'окон']},
            'E06': {'name': 'Оконные и дверные конструкции',
                   'keywords': ['окон', 'жалюз', 'ставн']},
            'E21': {'name': 'Бурение; горное дело',
                   'keywords': ['бурен', 'шахт', 'скважин']}
        }
    },
    'F': {
        'name': 'Машиностроение; освещение; отопление; двигатели',
        'subclasses': {
            'F01': {'name': 'Двигатели внутреннего сгорания',
                   'keywords': ['двигател', 'цилиндр', 'карбюратор']},
            'F02': {'name': 'Газотурбинные установки',
                   'keywords': ['турбин', 'реактивн', 'компрессор']},
            'F03': {'name': 'Двигатели на жидком топливе',
                   'keywords': ['гидравлич', 'волнов', 'приливн']},
            'F04': {'name': 'Насосы',
                   'keywords': ['насос', 'компрессор', 'вентилятор']},
            'F15': {'name': 'Гидравлика, пневматика',
                   'keywords': ['гидравл', 'пневмат', 'цилиндр']},
            'F16': {'name': 'Детали машин',
                   'keywords': ['подшипник', 'муфт', 'редуктор']},
            'F17': {'name': 'Хранение газов',
                   'keywords': ['газ', 'баллон', 'хранилищ']},
            'F21': {'name': 'Освещение',
                   'keywords': ['ламп', 'светильник', 'освещен']},
            'F22': {'name': 'Паровые котлы',
                   'keywords': ['котел', 'паров', 'топк']},
            'F23': {'name': 'Горение',
                   'keywords': ['горелк', 'топк', 'печ']},
            'F24': {'name': 'Отопление; вентиляция',
                   'keywords': ['отоплен', 'вентиляц', 'кондиционер']},
            'F25': {'name': 'Холодильная техника',
                   'keywords': ['холодильн', 'морозильн', 'конденсатор']},
            'F26': {'name': 'Сушка',
                   'keywords': ['сушк', 'сушилк', 'обезвоживан']},
            'F27': {'name': 'Печи; печные устройства',
                   'keywords': ['печ', 'термообработк', 'нагреван']},
            'F28': {'name': 'Теплообмен',
                   'keywords': ['теплообмен', 'радиатор', 'охлажден']},
            'F41': {'name': 'Оружие',
                   'keywords': ['оружи', 'пулемет', 'пистолет']},
            'F42': {'name': 'Боеприпасы; взрывные работы',
                   'keywords': ['боеприпас', 'взрывчат', 'гранат']}
        }
    },
    'G': {
        'name': 'Физика',
        'subclasses': {
            'G01': {'name': 'Измерительная техника',
                   'keywords': ['измерен', 'датчик', 'прибор']},
            'G02': {'name': 'Оптика',
                   'keywords': ['оптич', 'линз', 'микроскоп']},
            'G03': {'name': 'Фотография',
                   'keywords': ['фото', 'камер', 'пленк']},
            'G04': {'name': 'Измерение времени',
                   'keywords': ['час', 'хронограф', 'таймер']},
            'G05': {'name': 'Управление; регулирование',
                   'keywords': ['регулятор', 'управлен', 'автомат']},
            'G06': {'name': 'Вычислительная техника',
                   'keywords': ['вычисл', 'компьютер', 'процессор']},
            'G07': {'name': 'Учетные устройства',
                   'keywords': ['счетчик', 'касс', 'регистрац']},
            'G08': {'name': 'Сигнализация',
                   'keywords': ['сигнал', 'тревог', 'оповещен']},
            'G09': {'name': 'Образование; криптография',
                   'keywords': ['обучен', 'шифрован', 'дисплей']},
            'G10': {'name': 'Музыкальные инструменты',
                   'keywords': ['музык', 'инструмент', 'звук']},
            'G11': {'name': 'Хранение информации',
                   'keywords': ['памят', 'носител', 'запис']},
            'G12': {'name': 'Точная механика',
                   'keywords': ['точн', 'механизм', 'регулировк']},
            'G21': {'name': 'Ядерная физика',
                   'keywords': ['ядерн', 'реактор', 'радиоактив']}
        }
    },
    'H': {
        'name': 'Электричество',
        'subclasses': {
            'H01': {'name': 'Электротехника',
                   'keywords': ['электрич', 'реле', 'контактор']},
            'H02': {'name': 'Генерация, преобразование энергии',
                   'keywords': ['генератор', 'трансформатор', 'электродвиг']},
            'H03': {'name': 'Электронные схемы',
                   'keywords': ['схем', 'усилител', 'фильтр']},
            'H04': {'name': 'Телекоммуникации',
                   'keywords': ['связ', 'антенн', 'радио']},
            'H05': {'name': 'Специальные электротехнические устройства',
                   'keywords': ['электротехн', 'нагреван', 'рентген']}
        }
    }
}
 # Специальные правила для сложных случаев
        self.compiled_special_rules = [
            (re.compile(pattern, re.IGNORECASE), ipc_code)
            for pattern, ipc_code in [
                (r'\bробот|\bавтомат\w*', 'B25J'),
                (r'\bии\b|\bискусственн.*интеллект', 'G06N')
            ]
        ]

        # Стоп-слова для фильтрации
        self.stop_words = {'устройств', 'способ', 'метод', 'систем'}

        # Предварительная компиляция ключевых слов
        self.keyword_patterns = {}
        for section in self.ipc_hierarchy.values():
            for subclass_code, subclass_data in section['subclasses'].items():
                patterns = [
                    re.compile(rf'\b{keyword}', re.IGNORECASE)
                    for keyword in subclass_data['keywords']
                ]
                self.keyword_patterns[subclass_code] = patterns

    def preprocess_text(self, text):
        if pd.isna(text):
            return ""
        text = str(text).lower()
        text = re.sub(r'[^\w\s]', ' ', text)  # Удаление пунктуации
        text = re.sub(r'\d+', '', text)  # Удаление цифр
        return " ".join(word for word in text.split()
                       if len(word) > 3 and word not in self.stop_words)

    def find_ipc_subclass(self, text):
        processed_text = self.preprocess_text(text)

        # Проверка специальных правил
        for pattern, ipc_code in self.compiled_special_rules:
            if pattern.search(processed_text):
                return ipc_code

       # Основной алгоритм классификации
        scores = Counter()
        for subclass_code, patterns in self.keyword_patterns.items():
            for pattern in patterns:
                if pattern.search(processed_text):
                    scores[subclass_code] += 1

        return scores.most_common(1)[0][0] if scores else None

    #Получим названия подкласса по коду
    def get_subclass_name(self, ipc_code):
        if not ipc_code or pd.isna(ipc_code):
            return "Не определено"

        for section in self.ipc_hierarchy.values():
            if ipc_code in section['subclasses']:
                return section['subclasses'][ipc_code]['name']
        return "Неизвестный подкласс"

    #Добавим колонки
    def add_classification_columns(self, df, text_column='object name'):
        print("Начало классификации патентов...")

        # Создадим копию DataFrame
        result_df = df.copy()

        # Добавим колонку с кодом подкласса
        result_df['IPC_Subclass'] = result_df[text_column].apply(self.find_ipc_subclass)

        # Добавим колонку с названием подкласса
        result_df['IPC_Subclass_Name'] = result_df['IPC_Subclass'].apply(self.get_subclass_name)

        # Статистика выполнения
        print(f"Обработано патентов: {len(result_df)}")
        print(f"Успешно классифицировано: {result_df['IPC_Subclass'].notna().sum()}")

        return result_df

# Пример использования
if __name__ == "__main__":
    # Инициализация классификатора
    classifier = PatentClassifier()

    # Проверка названий колонок
    print("Доступные колонки:", cleaned_df.columns.tolist())

    # Добавим колонок классификации
    result_df = classifier.add_classification_columns(
        cleaned_df,
        text_column='object name'
    )


Доступные колонки: ['registration number', 'registration date', 'application number', 'application date', 'authors', 'authors in latin', 'patent holders', 'correspondence address', 'correspondence address in latin', 'object name', 'patent starting date', 'receipt date of additional data to application', 'date of application to which additional data has been received', 'number of application to which additional data has been received', 'initial application number', 'initial application date', 'initial application priority date', 'previous application number', 'previous application date', 'paris convention priority number', 'paris convention priority date', 'paris convention priority country code', 'patent grant publish date', 'patent grant publish number', 'revoked patent number', 'expiration date', 'actual', 'source_file', 'mpk_sections', 'mpk_section_name']
Начало классификации патентов...
Обработано патентов: 1135794
Успешно классифицировано: 621314


In [ ]:
result_df

,registration number,registration date,application number,application date,authors,authors in latin,patent holders,correspondence address,correspondence address in latin,object name,patent starting date,receipt date of additional data to application,date of application to which additional data has been received,number of application to which additional data has been received,initial application number,initial application date,initial application priority date,previous application number,previous application date,paris convention priority number,paris convention priority date,paris convention priority country code,patent grant publish date,patent grant publish number,revoked patent number,expiration date,actual,source_file,mpk_sections,mpk_section_name,IPC_Subclass,IPC_Subclass_Name
0,﻿1,25.06.1994,93030652.0,01.07.1993,Чалаганидзе Ш.И. Кокая Г.Г.,,Напетваридзе с.о.чалаганидзе шота иванович,NaN,NaN,Устройство для облесения крутых горных склонов,01.07.1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.06.1994,NaN,NaN,NaN,True,ПМОД.csv,E,Строительство и горное дело,None,Не определено
1,2,25.06.1994,93015739.0,24.03.1993,Дронов Владимир Дмитриевич Колобаев Владимир Никифорович Прохоров Александр Андреевич Лукьянченко Павел Петрович Столяров Евгений Александрович Головко Алексей Григорьевич Шорников Игорь Евгеньевич Савкин Михаил Николаевич,,Дронов владимир дмитриевич колобаев владимир никифорович прохоров александр андреевич лукьянченко павел петрович столяров евгений александрович головко алексей григорьевич шорников игорь евгеньевич савкин михаил николаевич,NaN,NaN,Регулятор вакуума,24.03.1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.06.1994,NaN,NaN,NaN,True,ПМОД.csv,None,Не определено,G05,Управление; регулирование
2,3,25.06.1994,93015868.0,25.03.1993,Звонов Б.М. Терсков Н.Г. Валиков А.В.,,"Малое производственное предприятие ""Пирс""",NaN,NaN,Устройство для отпугивания птиц,25.03.1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.06.1994,NaN,NaN,NaN,True,ПМОД.csv,None,Не определено,None,Не определено
3,4,25.06.1994,5061235.0,02.09.1992,Селиванов Виктор Викторович,NaN,Селиванов Виктор Викторович,NaN,NaN,Устройство для тренировки аккомодационной мышцы,02.09.1992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.06.1994,NaN,NaN,NaN,True,ПМОД.csv,None,Не определено,None,Не определено
4,5,25.06.1994,92008521.0,25.11.1992,Васильев Михаил Григорьевич,NaN,Васильев Михаил Григорьевич,NaN,NaN,Прибор для акупунктуры,25.11.1992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.06.1994,NaN,NaN,NaN,True,ПМОД.csv,G,Физика,G01,Измерительная техника
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1135789,141705,25.04.2024,2023504523.0,25.10.2023,Кучин Семен Геннадьевич,NaN,Кучин Семен Геннадьевич,"640002, г. Курган, п. Увал, ул. Трактовая, д. 1, корп. 19, кв. 172, Кучину С.Г.",NaN,КОНВЕРТ-УПАКОВКА ДЛЯ КОМПАКТ-ДИСКОВ,25.10.2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.04.2024,5.0,NaN,25.10.2028,True,ПООД.csv,B,Различные технологические процессы; транспортирование,B31,Изготовление бумаги
1135790,141706,25.04.2024,2023504951.0,06.10.2023,Сайгитов Махач Тимирбулатович,NaN,Сайгитов Махач Тимирбулатович,"196084, Санкт- Петербург, Московский проспект, д. 75, стр. А, а/я 143, Рассохиной Н.А.",NaN,ДИФФУЗОР ДЛЯ ПОТОЛКА,06.10.2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.04.2024,5.0,NaN,06.10.2028,True,ПООД.csv,None,Не определено,None,Не определено
1135791,141707,25.04.2024,2023505528.0,08.11.2023,Кучин Семен Геннадьевич,NaN,Кучин Семен Геннадьевич,"640002, г. Курган, п. Увал, ул. Трактовая, д. 1/19, кв. 172, Кучин С.Г.",NaN,ПРИКЛАД ДЛЯ ПИСТОЛЕТА,08.11.2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.04.2024,5.0,NaN,08.11.2028,True,ПООД.csv,None,Не определено,F41,Оружие
1135792,141708,26.04.2024,2023505924.0,28.11.2023,Новиков Александр Владимирович,NaN,"Общество с ограниченной ответственностью ""Нова-строй""","659309, Алтайский край, г. Бийск, пер. Уральск